In [ ]:
import random
import numpy as np

τ0 = 0.01
m = 7
α = 1
β = 2
ρ = 0.5
Pthreshold = 0.6
A = 1
Q = 1
penalty_value = 5
num_nodes = 5
iterations = 150
mixed_probabilities = False  

def generate_random_graph(num_nodes, mixed_probabilities=False):
    nodes = list(range(num_nodes))
    edges = [(i, j) for i in range(num_nodes) for j in range(num_nodes) if i != j]

    
    travel_time_matrix = {edge: random.randint(1, 30) for edge in edges}
    travel_time_matrix[(0, 0)] = 0


    if mixed_probabilities:
        probabilities = {node: random.choice([0.1, 1]) for node in nodes}
    else:
        probabilities = {node: random.uniform(0.1, 1) for node in nodes}

    deadlines = {node: random.randint(5, 110) for node in nodes}

    return nodes, edges, travel_time_matrix, probabilities, deadlines

nodes, edges, travel_time_matrix, probabilities, deadlines = generate_random_graph(
    num_nodes=num_nodes, mixed_probabilities=mixed_probabilities
)

In [ ]:


def calculate_transition_probability(τ, probabilities, tabu_list, α, β, k, Pthreshold, A, current_node, is_levy):
    probabilities_for_edges = {}
    tabu_set = set(tabu_list[k])
    denominator = sum(
        (τ[i, j] ** α) * (probabilities[j] ** β)
        for (i, j) in τ.keys()
        if i == current_node and j not in tabu_set
    )
    for (i, j), tau in τ.items():
        if i == current_node and j not in tabu_set:
            numerator = (tau ** α) * (probabilities[j] ** β)
            P_now = numerator / denominator
            
            
            P_levy = random.uniform(0, 1)  
            P_new = levy_mechanism(P_now, P_levy, Pthreshold, A, is_levy)
            
            probabilities_for_edges[(i, j)] = P_new

   
    total_probability = sum(probabilities_for_edges.values())
    for key in probabilities_for_edges:
        probabilities_for_edges[key] /= total_probability

    return probabilities_for_edges



def levy_mechanism(P_now, P_levy, Pthreshold, A, is_levy):
    if (P_levy >= Pthreshold) and (is_levy):
        S_new = (1 - A) * (1 - P_levy) / (1 - Pthreshold)
        return 1 - S_new * (1 - P_now)
    else:
        return P_now

def select_next_node(probabilities_for_edges):
    
    cumulative_probabilities = []
    cumulative_sum = 0
    for edge, probability in probabilities_for_edges.items():
        cumulative_sum += probability
        cumulative_probabilities.append((edge[1], cumulative_sum))

    
    random_value = random.uniform(0, 1)
    for node, cumulative_probability in cumulative_probabilities:
        if random_value <= cumulative_probability:
            return node

def calculate_pheromone_delta(ant_tours, cycle_cost, penalties, Q):
    delta_τ = {}
    for tour in ant_tours:
        for i in range(len(tour) - 1):
            edge = (tour[i], tour[i + 1])
            if edge not in delta_τ:
                delta_τ[edge] = Q / (cycle_cost + penalties)
            else:
                delta_τ[edge] += Q / (cycle_cost + penalties)
    return delta_τ

def calculate_expected_values(tour, travel_time_matrix, probabilities, deadlines, penalty_value):
    
    n = len(tour)
    E_travel_time = 0
    E_penalties = 0
    cumulative_travel_time = 0  

    for j in range(1, n):
        prod_part = np.prod([(1 - probabilities[tour[k]]) for k in range(1, j)])
        travel_time = travel_time_matrix[(tour[0], tour[j])]
        E_travel_time += probabilities[tour[j]] * travel_time * prod_part

    for i in range(1, n - 1):
        for j in range(i + 1, n):
            prod_part = np.prod([(1 - probabilities[tour[k]]) for k in range(i + 1, j)])
            travel_time = travel_time_matrix[(tour[i], tour[j])]
            E_travel_time += probabilities[tour[i]] * probabilities[tour[j]] * travel_time * prod_part


    for i in range(1, n):
        prod_part = np.prod([(1 - probabilities[tour[k]]) for k in range(i + 1, n)])
        travel_time = travel_time_matrix[(tour[i], tour[0])]
        E_travel_time += probabilities[tour[i]] * travel_time * prod_part

    for i in range(1, len(tour)):
       
        cumulative_travel_time += travel_time_matrix[(tour[i - 1], tour[i])]

        if cumulative_travel_time > deadlines[tour[i]]:
            E_penalties += probabilities[tour[i]] * penalty_value

    return E_travel_time, E_penalties


def ant_colony_optimization_with_levy(nodes, edges, travel_time_matrix, probabilities, deadlines, iterations, is_levy):
    τ = {edge: τ0 for edge in edges}
    best_solution = None
    best_cost = float('inf')

    for t in range(iterations):
        ant_tours = []
        for k in range(m): 
            tabu_list = {k: [0]}  
            current_node = 0

            for _ in range(len(nodes) - 1):
                probabilities_for_edges = calculate_transition_probability(τ, probabilities, tabu_list, α, β, k, Pthreshold, A, current_node, is_levy)
                next_node = select_next_node(probabilities_for_edges)
                tabu_list[k].append(next_node)
                current_node = next_node

            tabu_list[k].append(0)
            ant_tours.append(tabu_list[k])

        for edge in edges:
            τ[edge] = (1 - ρ) * τ[edge]

        for k, tour in enumerate(ant_tours):  
            E_travel_time, E_penalties = calculate_expected_values(tour, travel_time_matrix, probabilities, deadlines, penalty_value)
            cycle_cost = E_travel_time + E_penalties
            
            if cycle_cost < best_cost:
                best_cost = cycle_cost
                best_solution = tour

            for i in range(len(tour) - 1):
                edge = (tour[i], tour[i + 1])
                τ[edge] += Q / cycle_cost

    return best_solution, best_cost


is_levy = True
best_solution, best_cost = ant_colony_optimization_with_levy(
    nodes, edges, travel_time_matrix, probabilities, deadlines, iterations, is_levy
)
print("Best Solution:", best_solution)
print("Best Cost with levy:", best_cost)
print()

is_levy = False
best_solution, best_cost = ant_colony_optimization_with_levy(
    nodes, edges, travel_time_matrix, probabilities, deadlines, iterations, is_levy
)
print("Best Solution:", best_solution)
print("Best Cost without levy:", best_cost)


